<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_4/02_gansynth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GANSynthによる作曲
「GANSynth」により、MIDIデータに楽器の音声を付与します。  
生成には時間がかかるので、「編集」→「ノートブックの設定」→「ハードウェア アクセラレータ」で「GPU」を選択しておきましょう。  
このノートブックのコードは、以下のリンク先のコードを参考にしています。  
http://goo.gl/magenta/gansynth-demo

## ライブラリのインストール
Magentaと共に、音楽生成用のライブラリpyFluidSynth、MIDIデータを処理するためのpretty_midiなどをインストールします。

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

## ライブラリの導入
Magentaの必要な機能と、NumPyなどのライブラリを導入します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import tensorflow.compat.v1 as tf
import librosa

import magenta.music as mm
from magenta.models.gansynth.lib import flags as lib_flags
from magenta.models.gansynth.lib import generate_util as gu
from magenta.models.gansynth.lib import model as lib_model
from magenta.models.gansynth.lib import util
from note_seq.notebook_utils import colab_play as play

import note_seq

## 各設定値
曲の生成に関する各値を設定します。

In [ ]:
BATCH_SIZE = 16  # 一度に扱うデータ数
SR = 16000  # サンプリングレート

## 関数の設定
音声を処理する関数です。  
以下のリンク先のコードを使用しています。  
http://goo.gl/magenta/gansynth-demo

In [ ]:
def load_midi(midi_path, min_pitch=36, max_pitch=84):
  """Load midi as a notesequence."""
  midi_path = util.expand_path(midi_path)
  ns = note_seq.midi_file_to_sequence_proto(midi_path)
  pitches = np.array([n.pitch for n in ns.notes])
  velocities = np.array([n.velocity for n in ns.notes])
  start_times = np.array([n.start_time for n in ns.notes])
  end_times = np.array([n.end_time for n in ns.notes])
  valid = np.logical_and(pitches >= min_pitch, pitches <= max_pitch)
  notes = {'pitches': pitches[valid],
           'velocities': velocities[valid],
           'start_times': start_times[valid],
           'end_times': end_times[valid]}
  return ns, notes

def get_envelope(t_note_length, t_attack=0.010, t_release=0.3, sr=16000):
  """Create an attack sustain release amplitude envelope."""
  t_note_length = min(t_note_length, 3.0)
  i_attack = int(sr * t_attack)
  i_sustain = int(sr * t_note_length)
  i_release = int(sr * t_release)
  i_tot = i_sustain + i_release  # attack envelope doesn't add to sound length
  envelope = np.ones(i_tot)
  # Linear attack
  envelope[:i_attack] = np.linspace(0.0, 1.0, i_attack)
  # Linear release
  envelope[i_sustain:i_tot] = np.linspace(1.0, 0.0, i_release)
  return envelope

def combine_notes(audio_notes, start_times, end_times, velocities, sr=16000):
  """Combine audio from multiple notes into a single audio clip.

  Args:
    audio_notes: Array of audio [n_notes, audio_samples].
    start_times: Array of note starts in seconds [n_notes].
    end_times: Array of note ends in seconds [n_notes].
    sr: Integer, sample rate.

  Returns:
    audio_clip: Array of combined audio clip [audio_samples]
  """
  n_notes = len(audio_notes)
  clip_length = end_times.max() + 3.0
  audio_clip = np.zeros(int(clip_length) * sr)

  for t_start, t_end, vel, i in zip(start_times, end_times, velocities, range(n_notes)):
    # Generate an amplitude envelope
    t_note_length = t_end - t_start
    envelope = get_envelope(t_note_length)
    length = len(envelope)
    audio_note = audio_notes[i, :length] * envelope
    # Normalize
    audio_note /= audio_note.max()
    audio_note *= (vel / 127.0)
    # Add to clip buffer
    clip_start = int(t_start * sr)
    clip_end = clip_start + length
    audio_clip[clip_start:clip_end] += audio_note

  # Normalize
  audio_clip /= audio_clip.max()
  audio_clip /= 2.0
  return audio_clip

# Plotting tools
def specplot(audio_clip):
  p_min = np.min(36)
  p_max = np.max(84)
  f_min = librosa.midi_to_hz(p_min)
  f_max = 2 * librosa.midi_to_hz(p_max)
  octaves = int(np.ceil(np.log2(f_max) - np.log2(f_min)))
  bins_per_octave = 36
  n_bins = int(bins_per_octave * octaves)
  C = librosa.cqt(audio_clip, sr=SR, hop_length=2048, fmin=f_min, n_bins=n_bins, bins_per_octave=bins_per_octave)
  power = 10 * np.log10(np.abs(C)**2 + 1e-6)
  plt.matshow(power[::-1, 2:-2], aspect='auto', cmap=plt.cm.magma)
  plt.yticks([])
  plt.xticks([])

## GANSynthのモデル
パスを指定し、GANSynthの学習済みモデルを読み込みます。

In [ ]:
tf.disable_v2_behavior()  # tensorflow2で1.xのコードを動かす
tf.reset_default_graph()  # tensorflowのグラフをリセット

model_dir = "gs://magentadata/models/gansynth/acoustic_only"
flags = lib_flags.Flags({
    "batch_size_schedule": [BATCH_SIZE],
    "tfds_data_dir": "gs://tfds-data/datasets",
})
model = lib_model.Model.load_from_path(model_dir, flags)

「01_multitrack_musicvae.ipynb」で作成した、「conditional_vae.mid」を、ノートブック左の「ファイル」にアップロードしましょう。  
以下は、このファイルを読み込むコードです。  

In [ ]:
midi_path = "conditional_vae.mid"
ns, notes = load_midi(midi_path)

note_seq.plot_sequence(ns)
note_seq.play_sequence(ns, synth=note_seq.fluidsynth) 

GANSynthにより、読み込んだMIDIデータに楽器の音声を付与します。   
楽器はランダムにゆっくりと切り替わります。  

In [ ]:
seconds_per_instrument = 5  # 楽器が切り替わる間隔

# 潜在変数がランダムにゆっくりと変化
z_instruments, t_instruments = gu.get_random_instruments(  # 潜在変数とその時間
    model,
    notes["end_times"][-1],
    secs_per_instrument=seconds_per_instrument)

# 各noteの潜在変数を取得
z_notes = gu.get_z_notes(notes["start_times"], z_instruments, t_instruments)

# 各ノートの音声を生成
audio_notes = model.generate_samples_from_z(z_notes, notes["pitches"])

# 1つの音声にまとめる
audio = combine_notes(
    audio_notes,
    notes["start_times"],
    notes["end_times"],
    notes["velocities"]
    )

specplot(audio)  # スペクトログラムの表示
play(audio, sample_rate=SR)

音声ををwavデータに変換し、保存してダウンロードします。


In [ ]:
file_name = "gansynth.wav"
gu.save_wav(audio, file_name)
files.download(file_name)